### To hide cells, use the code below, add the "hide_toggle()" to a cell to enable it.

### Another method is using tags and running: 

jupyter nbconvert Projeto_Final.ipynb --TagRemovePreprocessor.remove_input_tags='{"remove"}'

on the terminal.

https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook

In [14]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)
hide_toggle()

# ICD - Projeto Final: Gun Violence

## Introdução

Neste trabalho vamos analisar características de incidentes com armas.

## Motivação

A motivação pelo trabalho veio de uma questão polêmica atual no Brasil sobre a legalização de armas. 
Para entender melhor o que o porte pode trazer, resolveu-se analisar dados de um país que tenha porte de arma liberado, os Estados Unidos da América, sendo assim será utilizada uma base que registra incidentes com arma de fogo através dos anos, começando em 2013 e seguindo até 2018.

## Dados Utilizados

Para a realização deste projeto, foi necessário utilizar alguns conjuntos de dados de domínio público. Nesta seção iremos entrar em mais detalhes sobre as bases utilizadas.

In [13]:
import pandas as pd
import re
hide_toggle()

### Gun Violence

DESCRIÇÃO

Fonte: [Gun Violence Data - James Ko](https://www.kaggle.com/jameslko/gun-violence-data)

In [ ]:
# Load gun violence dataset
gun_violence = pd.read_csv('../databases/gun_violence.zip', compression='zip')

### State Population Totals and Components of Change: 2010-2018

DESCRIÇÃO

Fonte: [United States Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html)

In [ ]:
# Load population dataset
population = pd.read_csv('../databases/nst-est2018-alldata.zip', compression='zip')

## Limpeza dos Dados

As bases de dados selecionadas possuem alguns dados nos quais não estamos interessados e que podemos remover para melhorar o desempenho e facilitar o trabalho.
Além disso algumas estão faltando informações ou possuem uma formatação não muito adequada

### Gun Violence

In [ ]:
# Drop unnecessary columns
gun_violence = gun_violence[[
    'incident_id',
    'date',
    'state',
    'latitude',
    'longitude',
    'n_killed',
    'n_injured',
    'gun_stolen',
    'gun_type',
    'n_guns_involved',
    'participant_age',
    'participant_age_group',
    'participant_gender',
    'participant_type'
]]

In [ ]:
# Drop missing values
gun_violence.dropna(inplace=True)

In [ ]:
# Change date formatting
gun_violence['year'] = gun_violence['date'].map(lambda x: x[0:4])
gun_violence['month'] = gun_violence['date'].map(lambda x: x[5:7])
gun_violence['day'] = gun_violence['date'].map(lambda x: x[8:10])
gun_violence.drop(columns='date', inplace=True)

In [ ]:
# Change formatting on gun_stolen, gun_type, participant_age, etc.
gun_violence['gun_stolen'] = gun_violence['gun_stolen'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['gun_type'] = gun_violence['gun_type'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_age'] = gun_violence['participant_age'].map(lambda x: re.findall(r'[0-9]+', x)[1::2])
gun_violence['participant_age_group'] = gun_violence['participant_age_group'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_gender'] = gun_violence['participant_gender'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_type'] = gun_violence['participant_type'].map(lambda x: re.findall(r'[a-zA-Z]+', x))

Vamos criar um novo data frame com informações dos participantes dos incidentes para podermos trabalhar melhor.

In [ ]:
# Create new Data Frame for participants
participants = []
for index, row in gun_violence.iterrows():
    for age, group, gender, p_type in zip(row['participant_age'], row['participant_age_group'], row['participant_gender'], row['participant_type']):
        participants.append([row['incident_id'], age, group, gender, p_type])

participants = pd.DataFrame(participants, columns=['incident_id', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_type'])

In [ ]:
participants.head()

### Population

In [ ]:
# Drop unnecessary columns
population = population[[
    'NAME',
    'POPESTIMATE2013',
    'POPESTIMATE2014',
    'POPESTIMATE2015',
    'POPESTIMATE2016',
    'POPESTIMATE2017',
    'POPESTIMATE2018'
]]

# Rename columns
population.columns = ['state', '2013', '2014', '2015', '2016', '2017', '2018']

print(population.shape)

## Análise Exploratória

In [ ]:
alt.data_transformers.enable('default', max_rows=None)

### Número de Incidentes por Estado

In [ ]:
# TODO: Divide the number of incides by the population to get the number of incidents per capita.

states = alt.topo_feature(data.us_10m.url, 'states')

# US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')

points = alt.Chart(gun_violence).transform_aggregate(
    latitude='mean(latitude)',
    longitude='mean(longitude)',
    count='count()',
    groupby=['state']
).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('count:Q', title='Number of Incidents'),
    color=alt.value('steelblue'),
    tooltip=['state:N','count:Q']
).properties(
    title='Number of gun incidents per state in the US'
)

background + points

# alt.Chart(gun_violence).mark_bar().encode(
#     alt.X('state', axis=alt.Axis(title='Estado')),
#     alt.Y('count()', axis=alt.Axis(title='Número de Registros'))
# )

### Distribuição de Incidentes ao Longo dos Anos

In [ ]:
alt.Chart(gun_violence).mark_rect().encode(
    alt.X('month', axis=alt.Axis(title='Mês')),
    alt.Y('year', sort='descending', axis=alt.Axis(title='Ano')),
    alt.Color('count()')
).properties(
    width=500,
    height=300
)

### Idade dos envolvidos

In [ ]:
alt.Chart(participants).mark_bar().encode(
    alt.X('participant_age:Q', bin=alt.BinParams(extent=[0,100], step=5), title='Idade'),
    alt.Y('count()', title='Frequência'),
    color='participant_type'
).properties(
    width=500,
    height=300
)

## Testes de Hipótese

### Teste 1

### Teste 2

## Regressão

## Classificação